<a href="https://colab.research.google.com/github/Abhieo07/Basics_ml/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment Analysis

**Movie Review Dataset**

We'll start by loading in the IMDB movie review dataset from keras. This dataset contain 25,000 review from IMDB where each one is already preprocessed and has a label either positive or negative.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_label), (test_data, test_label) = imdb.load_data(num_words=VOCAB_SIZE)

17464789/17464789 [==============================] - 0s 0us/step


In [2]:
# Let's look at one review
train_data[0]
# length of data is not unique

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

**More preprocessing**

Since length of data is not unique, it raises an issue we cannot pass different length data into our neural network. So, we have to make each review of same length.

* if the review is greater than 250 words then trim off the extra words.

* if the review is less than 250 words add neccessary amount of 0's to make it equal to 250.




In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

**Creating the Model**

We'll use word embedding layer as a first layer in our model and add a LSTM layer afterwards that feeds into a dense mode to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


**Training**

Now it's time to compile and train the mode

In [6]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['acc'])

history = model.fit(train_data, train_label, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 84s 128ms/step - loss: 0.4491 - acc: 0.7813 - val_loss: 0.3227 - val_acc: 0.8672
Epoch 2/10
625/625 [==============================] - 81s 130ms/step - loss: 0.2568 - acc: 0.9028 - val_loss: 0.2837 - val_acc: 0.8850
Epoch 3/10
625/625 [==============================] - 78s 125ms/step - loss: 0.1979 - acc: 0.9279 - val_loss: 0.3395 - val_acc: 0.8720
Epoch 4/10
625/625 [==============================] - 87s 140ms/step - loss: 0.1618 - acc: 0.9425 - val_loss: 0.2999 - val_acc: 0.8812
Epoch 5/10
625/625 [==============================] - 82s 132ms/step - loss: 0.1345 - acc: 0.9549 - val_loss: 0.2988 - val_acc: 0.8832
Epoch 6/10
625/625 [==============================] - 94s 151ms/step - loss: 0.1109 - acc: 0.9615 - val_loss: 0.3157 - val_acc: 0.8828
Epoch 7/10
625/625 [==============================] - 85s 136ms/step - loss: 0.0949 - acc: 0.9683 - val_loss: 0.3855 - val_acc: 0.8640
Epoch 8/10
625/625 [==============================] - 8

In [7]:
result = model.evaluate(test_data, test_label)
print(result)

782/782 [==============================] - 34s 43ms/step - loss: 0.4396 - acc: 0.8588
[0.4396248161792755, 0.8587599992752075]


**Making Prediction**

Now let's use our network to make prediction on our own reviews.

In [9]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = 'that movie was just amazing, so amazing'
encode = encode_text(text)
print(encode)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [10]:
# while were at it let's make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num]+" "

  return text[:-1]

print(decode_integers(encode))

that movie was just amazing so amazing


In [13]:
# now to make prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so ! I really loved it and would watch it again because it was amazingly freat"
predict(positive_review)

negative_review = "that movie sucked. I hated it and wouldn't watch again. was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 36ms/step
[0.8188943]
1/1 [==============================] - 0s 32ms/step
[0.12063371]
